In [2]:
import pandas as pd

train_set_raw = pd.read_csv("./../data/train.csv", index_col='id')
test_set_raw = pd.read_csv("./../data/test.csv", index_col='id')
train_set_raw

,review,sentiment
id,,
0,Action & Adventure.Billie Clark is twenty year...,1
1,I found this early talkie difficult to watch a...,0
2,It was Libby talking to Desmond in the flashba...,1
3,We fans of Ed Wood tend to be an obsessive bun...,1
4,"Hmm, Hip Hop music to a period western. Modern...",0
...,...,...
39577,This story had a different angle that intrigue...,0
39578,I thought the movie (especially the plot) need...,0
39579,The title of this film nearly put me off watch...,1


In [3]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nikita_gordia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nikita_gordia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nikita_gordia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

def prepare(ds, params):
    shadow_vectorizer = CountVectorizer(
        tokenizer=nltk.word_tokenize,
        stop_words=stopwords.words('english'),
        ngram_range=params.get('ngram_range', (1, 1))
    )
    vectorizer = CountVectorizer(
        analyzer=params['analyzer_builder'](shadow_vectorizer.build_analyzer()),
        max_features=params.get('max_features', None))
    X = vectorizer.fit_transform(ds['review']).toarray()
    return X

def prepare_dataset(train_set_raw, params):
    return prepare(train_set_raw, params), train_set_raw['sentiment']

def simple_analyzer_builder(base_analyzer):
    def analyzer(doc):
        return [word.lower() for word in base_analyzer(doc)]
    return analyzer

In [28]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

def evaluate_preprocessing_on_model(model, cut_n, **params):
    X_train, y_train = prepare_dataset(train_set_raw[:cut_n], params)
    print(cross_val_score(model, X_train, y_train).mean())

evaluate_preprocessing_on_model(MultinomialNB(), 1000,
    analyzer_builder=simple_analyzer_builder)

/Users/nikita_gordia/opt/anaconda3/envs/HackathonTest/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


0.807


In [29]:
def stem_analyzer_builder(base_analyzer):
    stemmer = nltk.stem.snowball.EnglishStemmer()
    def analyzer(doc):
        return [stemmer.stem(word.lower()) for word in base_analyzer(doc)]
    return analyzer

evaluate_preprocessing_on_model(MultinomialNB(), 1000,
    analyzer_builder=stem_analyzer_builder)

/Users/nikita_gordia/opt/anaconda3/envs/HackathonTest/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


0.799


In [30]:
def lem_analyzer_builder(base_analyzer):
    stemmer = nltk.stem.WordNetLemmatizer()
    def analyzer(doc):
        return [stemmer.lemmatize(word.lower()) for word in base_analyzer(doc)]
    return analyzer

evaluate_preprocessing_on_model(MultinomialNB(), 1000,
    analyzer_builder=lem_analyzer_builder)

/Users/nikita_gordia/opt/anaconda3/envs/HackathonTest/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


0.8030000000000002


In [31]:
evaluate_preprocessing_on_model(MultinomialNB(), 100,
    analyzer_builder=lem_analyzer_builder,
    max_features=1000)

/Users/nikita_gordia/opt/anaconda3/envs/HackathonTest/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


0.7700000000000001


In [32]:
evaluate_preprocessing_on_model(MultinomialNB(), 1000,
    analyzer_builder=lem_analyzer_builder,
    max_features=2000,
    ngram_range=(1, 3))

/Users/nikita_gordia/opt/anaconda3/envs/HackathonTest/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


0.8230000000000001


In [23]:

evaluate_preprocessing_on_model(MultinomialNB(), 1000,
    analyzer_builder=lem_analyzer_builder,
    max_features=2050,
    ngram_range=(1, 3))

/Users/nikita_gordia/opt/anaconda3/envs/HackathonTest/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


0.8210000000000001


In [24]:
from sklearn.linear_model import LogisticRegression

evaluate_preprocessing_on_model(LogisticRegression(), 1000,
    analyzer_builder=lem_analyzer_builder,
    max_features=1000)

/Users/nikita_gordia/opt/anaconda3/envs/HackathonTest/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/Users/nikita_gordia/opt/anaconda3/envs/HackathonTest/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nikita_gordia/opt/anacond

0.7779999999999999


In [25]:
evaluate_preprocessing_on_model(LogisticRegression(), 1000,
    analyzer_builder=lem_analyzer_builder,
    max_features=2000,
    ngram_range=(1, 3))

/Users/nikita_gordia/opt/anaconda3/envs/HackathonTest/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/Users/nikita_gordia/opt/anaconda3/envs/HackathonTest/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nikita_gordia/opt/anacond

0.7969999999999999


In [26]:

evaluate_preprocessing_on_model(LogisticRegression(), 1000,
    analyzer_builder=lem_analyzer_builder,
    max_features=2050,
    ngram_range=(1, 3))

/Users/nikita_gordia/opt/anaconda3/envs/HackathonTest/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/Users/nikita_gordia/opt/anaconda3/envs/HackathonTest/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nikita_gordia/opt/anacond

0.793
